**Final Regression Model for CNN**

Model trained and tested on a total of 15k images of resolution 50x50x3.

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.constraints import maxnorm
from sklearn.model_selection import train_test_split
import numpy as np
import h5py
from pathlib import Path

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Prepare Image data**

In [4]:
hdf5_dir = Path("/content/drive/My Drive/")
def read_many_hdf5(num_images):
    """ Reads image from HDF5.
        Parameters:
        ---------------
        num_images   number of images to read

        Returns:
        ----------
        images      images array, (N, 32, 32, 3) to be stored
        labels      associated meta data, int label (N, 1)
    """
    images, labels = [], []

    # Open the HDF5 file
    file = h5py.File(hdf5_dir / f"{num_images}_.h5", "r+")

    images = np.array(file["/images"]).astype("uint8")
    labels = np.array(file["/confidence"]).astype("uint8")

    return images, labels

X_data, Y_data = read_many_hdf5(15000)
print(X_data.shape, Y_data.shape)

(15000, 50, 50, 3) (15000,)


In [0]:
X_data = X_data/255.0
Y_data = np.where(Y_data < 1, 1, Y_data)

In [0]:
#X, Y = create_data()

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.2)

In [10]:
maxConfidence = y_train.max()
print(maxConfidence)

96


In [0]:
y_train = y_train / maxConfidence
y_test = y_test / maxConfidence

In [0]:
### Creating Model
def create_model():
  model = Sequential()

  model.add(Conv2D(32, (3, 3), input_shape=(50, 50, 3), activation='relu', padding='same'))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())

  #2nd convolutional layer
  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())

  #3rd convolutional layer
  model.add(Conv2D(128, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())

  model.add(Flatten())
  model.add(Dropout(0.2))

  model.add(Dense(256, kernel_constraint=maxnorm(3)))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())
      
  model.add(Dense(128, kernel_constraint=maxnorm(3)))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())

  #final layer should have as many neurons as there are classes. Since we have one class and it is regression problem so the value of final neuron will be between 0-99
  model.add(Dense(1))

  optimizer = 'adam'

  model.compile(loss='mean_absolute_percentage_error', optimizer=optimizer)
  return model


In [0]:
modelCNN = create_model()

In [0]:
print(modelCNN.summary())

In [16]:
modelCNN.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=100, batch_size=20)



Train on 12000 samples, validate on 3000 samples
Epoch 1/100
12000/12000 [==============================] - 17s 1ms/step - loss: 735.4219 - val_loss: 421.0069
Epoch 2/100
12000/12000 [==============================] - 14s 1ms/step - loss: 273.1254 - val_loss: 1169.6513
Epoch 3/100
12000/12000 [==============================] - 14s 1ms/step - loss: 222.8041 - val_loss: 783.0967
Epoch 4/100
12000/12000 [==============================] - 14s 1ms/step - loss: 174.6318 - val_loss: 17099.2102
Epoch 5/100
12000/12000 [==============================] - 14s 1ms/step - loss: 162.8014 - val_loss: 213.1261
Epoch 6/100
12000/12000 [==============================] - 14s 1ms/step - loss: 163.1266 - val_loss: 3746.6853
Epoch 7/100
12000/12000 [==============================] - 14s 1ms/step - loss: 135.4540 - val_loss: 51626.0718
Epoch 8/100
12000/12000 [==============================] - 14s 1ms/step - loss: 126.3613 - val_loss: 246.1026
Epoch 9/100
12000/12000 [==============================] - 14s 

In [17]:
print("[INFO] predicting image confidences...")
preds = modelCNN.predict(X_test)

[INFO] predicting image confidences...


In [0]:
diff = preds.flatten() - y_test
percentDiff = (diff / y_test) * 100
absPercentDiff = np.abs(percentDiff)

In [0]:
mean = np.mean(absPercentDiff)
std = np.std(absPercentDiff)

In [21]:
print("[INFO] avg. confidence: {}, std confidence: {}".format(
	Y_data.mean(),
	Y_data.std()))
print("[INFO] mean: {:.2f}%, std: {:.2f}%".format(mean, std))

[INFO] avg. confidence: 71.57886666666667, std confidence: 33.123225185895706
[INFO] mean: 1866.40%, std: 25999.79%


Output

[INFO] avg. confidence: 71.9118, std confidence: 32.74083109452171
[INFO] mean: 87.02%, std: 31.94%

========

[INFO] avg. confidence: 71.57886666666667, std confidence: 33.123225185895706
[INFO] mean: 1866.40%, std: 25999.79%